In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

from datetime import datetime, date, timedelta

from bokeh.layouts import layout, column, row
from bokeh.models import ColumnDataSource, GeoJSONDataSource, DateSlider, Button, Patches, Select, Range1d, LinearColorMapper
from bokeh.plotting import figure
from bokeh.palettes import Viridis6
from bokeh.io import show, output_notebook, curdoc

from data_handler import DataHandler

output_notebook()

Loading BokehJS ...

# ToDo:

* get other data and include it to show other details (tests, economy, unemployment, ...
* fix ranges for line graphs
* link views
* change the color by day in the animation by cases
* select different countrys and when clicked we only see the data for this country

In [2]:
def app(doc):
    # load data
    dh = DataHandler()
    source = ColumnDataSource(dh.initial_view())
    
    # create line plots   
    line_plot_1 = figure(plot_width=800, 
                   plot_height=200,
                   x_axis_type="datetime")
    
    line_plot_2 = figure(plot_width=800, 
                   plot_height=200,
                   x_axis_type="datetime")
    
    line_plot_3 = figure(plot_width=800, 
                   plot_height=200,
                   x_axis_type="datetime")

    line_plot_1.line('date', 'line_1', source=source)
    line_plot_2.line('date', 'line_2', source=source)
    line_plot_3.line('date', 'line_3', source=source)
    
    # set inditial optins
    SELECT_OPTIONS = dh.fields
    INITIAL_VIEW = [SELECT_OPTIONS[0], SELECT_OPTIONS[1]]
    
     # date ranges
    x_range_start = datetime(dh.date_range[0].year, dh.date_range[0].month, dh.date_range[0].day)
    x_range_end = datetime(dh.date_range[1].year, dh.date_range[1].month, dh.date_range[1].day)

    line_plot_1.x_range.start = x_range_start
    line_plot_1.x_range.end = x_range_end
    line_plot_2.x_range.start = x_range_start
    line_plot_2.x_range.end = x_range_end
    line_plot_3.x_range.start = x_range_start
    line_plot_3.x_range.end = x_range_end
    
    line_plot_1.y_range = Range1d(0, dh.y_range_end['EUR'][INITIAL_VIEW[0]])
    line_plot_2.y_range = Range1d(0, dh.y_range_end['EUR'][INITIAL_VIEW[1]])
    line_plot_3.y_range = Range1d(0, 100)
    
    
    # chloropleth
    TOOLS_chloropleth = 'hover,tap,pan'
    chloropleth = figure(plot_width=500, 
                         plot_height=400, 
                         tools=TOOLS_chloropleth, 
                         background_fill_color='#7dade0', 
                         background_fill_alpha = 0.9)
    chloropleth.grid.visible = False
    chloropleth.hover.tooltips = [
        ('Country', '@country'),
        ('Final Cumulative Cases', '@cases'),
        ('Final Cumulative Deaths', '@deaths'),
    ]
    geo_source = GeoJSONDataSource(geojson=dh.geo_data.to_json())
    palette = tuple(reversed(Viridis6))
    color_mapper = LinearColorMapper(palette=palette)
    patches = Patches(xs="xs", ys="ys",
                        fill_alpha=0.7, 
                        fill_color={'field': INITIAL_VIEW[0], 'transform': color_mapper},
                        line_color='white', 
                        line_width=0.3)
    chloropleth.add_glyph(geo_source, patches)
    
    # create interactions
    # slider
    def update(attrname, old, new):
        date = slider.value_as_datetime.date()
        source.data = dh.update_view(date, select1.value, select2.value, select_iso.value)
        line_plot_1.y_range.update(end=dh.y_range_end[select_iso.value][select1.value])
        line_plot_2.y_range.update(end=dh.y_range_end[select_iso.value][select2.value])
        line_plot_3.y_range.update(end=source.data.line_3.max())
        
        
    
    slider = DateSlider(start=dh.date_range[0], end=dh.date_range[1], value=dh.date_range[0], step=1, title="Date")
    slider.on_change('value', update)
    
    callback_id = None
    # automated play button
    def animate_update():
        global callback_id
        date = slider.value_as_datetime.date() + timedelta(days=1)
        if date >= dh.date_range[1]:
            date = dh.date_range[0]
        slider.value = date
    
    def animate():
        global callback_id
        if button.label == '► Play':
            button.label = '❚❚ Pause'
            callback_id = curdoc().add_periodic_callback(animate_update, 200)
        else:
            button.label = '► Play'
            curdoc().remove_periodic_callback(callback_id)
    
    button = Button(label='► Play', width=60)
    button.on_click(animate)
    
    def update_color(attrname, old, new):
        patches.update(fill_color={'field': select_color_by.value, 'transform': color_mapper})
        
    # Selectors
    CATEGORIES=['Governance and socio-economic measures', 'Lockdown',
       'Movement restrictions', 'Public health measures',
       'Social distancing', 'Humanitarian exemption']
    OPTIONS = ['cases', 'deaths']
    select1 = Select(title = 'Draw line plot 1 by:', value=INITIAL_VIEW[0], options=SELECT_OPTIONS)
    select2 = Select(title = 'Draw line plot 2 by:', value=INITIAL_VIEW[1], options=SELECT_OPTIONS)
    select3 = Select(title = 'Draw line plot 3 by Restriction category:', value=CATEGORIES[0], options=CATEGORIES)
    
    
    select1.on_change('value', update)
    select2.on_change('value', update)
    select3.on_change('value', update)
        
    iso_list = dh.iso_list
    iso_list.insert(0, 'EUR')
    select_iso = Select(title = 'Country:', value='EUR', options=iso_list)
    select_iso.on_change('value', update)
    
    select_color_by = Select(title='Color by:', value=INITIAL_VIEW[0], options=SELECT_OPTIONS)
    select_color_by.on_change('value', update_color)
    
    # create layout
    tools = column(button, slider, select_color_by, select_iso, select1, select2, select3)
    top = row(chloropleth, tools)
    bottom = column(line_plot_1, line_plot_2,line_plot_3)
    layout = column(top, bottom)
    
    # add to server
    doc.add_root(layout)

In [3]:
show(app)

ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PATCH-DOC' content: {'events': [{'kind': 'ModelChanged', 'model': {'id': '1147'}, 'attr': 'value', 'new': 'Cumulated Restrictions overall'}], 'references': []} 
 error: AttributeError("'PropertyValueColumnData' object has no attribute 'line_3'")
Traceback (most recent call last):
  File "/home/martin/.local/share/virtualenvs/visualize_covid-19-ZlIiDbw_/lib/python3.7/site-packages/bokeh/server/protocol_handler.py", line 90, in handle
    work = await handler(message, connection)
  File "/home/martin/.local/share/virtualenvs/visualize_covid-19-ZlIiDbw_/lib/python3.7/site-packages/bokeh/server/session.py", line 67, in _needs_document_lock_wrapper
    result = func(self, *args, **kwargs)
  File "/home/martin/.local/share/virtualenvs/visualize_covid-19-ZlIiDbw_/lib/python3.7/site-packages/bokeh/server/session.py", line 261, in _handle_patch
    message.apply_to_document(self.document, self)
  File "/home/martin/.l